### Importing Necessary Libraries, Setting Seed and Device

The below code imports the required libraries for the running of this notebook. Further we set seed to 42 to maintain consistency or reproducibility and set devide to cuda (if available)

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import random

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

DATA_PATH = '/kaggle/input/partc-data/partc_data.csv' # we used this path in kaggle. please change to whichever path partc_data.csv is stored in accordingly

Using device: cuda


### Defining the BERT-based classifier model


#### ***CaptionClassifier (model class)***
BERT-based classifier for identifying which model generated a caption. The constructor initializes the BERT model, dropout and additional classifier layers which we will be fine-tuning on our dataset from Part-B


forward() method returns the model **output logits**.



In [5]:
# BERT-based classifier model
class CaptionClassifier(nn.Module):
    def __init__(self, bert_model_name="bert-base-uncased", dropout_rate=0.1):
        super(CaptionClassifier, self).__init__()
        
        self.bert = BertModel.from_pretrained(bert_model_name)
        
        bert_hidden_size = self.bert.config.hidden_size # output dim from BERT

        # adding classifier layers
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Sequential(
            nn.Linear(bert_hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 2),
            nn.Softmax(dim=1)
        )
    
    def forward(self, input_ids, attention_mask, token_type_ids=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        
        # use the [CLS] token embedding (first token)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        
        probs = self.classifier(pooled_output)
        
        return probs

### Defining the CaptionDataset class to process data


#### ***CaptionDataset (Dataset class)***
Here we process the data that is input as a dataframe. The ground-truth captions, generated captions and perturbation (occlusion percentage) are to be input to the BERT classifier as:

> \<original_caption> \<SEP> \<generated_caption> \<SEP> \<perturbation_percentage>


**A: smolvlm** 

**B: custom model - CLIP-ViT encoder + GPT2 decoder + contrastive loss**


In [6]:
# To process the data from csv into a Dataset object for data-loading
class CaptionDataset(Dataset):
    def __init__(self, data_df, tokenizer, max_length=512):
        self.data_df = data_df
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.data_df)
    
    def __getitem__(self, idx):
        row = self.data_df.iloc[idx]
        
        original_caption = str(row['original_caption'])
        generated_caption = str(row['generated_caption'])
        perturbation = str(row['perturbation_percentage'])
        
        # Input Text: <original_caption> <SEP> <generated_caption> <SEP> <perturbation_percentage>
        input_text = original_caption + " " + self.tokenizer.sep_token + " " + generated_caption + " " + self.tokenizer.sep_token + " " + perturbation
        
        
        encoding = self.tokenizer(input_text, add_special_tokens=True, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')
        
        label = row['model_label']
        if label == 'A':  # Model A is smolvlm
            label_tensor = torch.tensor([1.0, 0.0], dtype=torch.float)
        else:  # Model B is our custom model (CLIP-ViT + GPT2 with contrastive loss)
            label_tensor = torch.tensor([0.0, 1.0], dtype=torch.float)
        
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'token_type_ids': encoding['token_type_ids'].squeeze(),
            'label': label_tensor
        }

### Defining the training function (with validation)


#### ***train_classifier (function)***
BERT-based classifier for identifying which model generated a caption. The constructor initializes the BERT model, dropout and additional classifier layers which we will be fine-tuning on our dataset from Part-B

 
Trains the BERT-based caption classifier.
- model (nn.Module): Custom image captioning model.
- dataloader (DataLoader): Training data loader.
- optimizer: Optimizer (e.g., Adam).
- criterion (Loss): Loss function.
- device (str): Device to use ('cuda' or 'cpu').
- epochs (int): Number of epochs.

In [7]:
# storing the loss and accuracy metrics here to maintain training history
history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }

def train_classifier(model, train_loader, val_loader, optimizer, criterion, device, epochs=5):
    # note, we included val_loader as well here since it is required for validatin in between training epochs. This is done in order to store the best model
    # and to use it for eval finally. You may remove this altogether and just use the train_loader, then use the final saved model to eval.
    best_val_acc = 0.0
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        train_progress = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs} [Train]')
        
        for batch in train_progress:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['label'].to(device)
            
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask, token_type_ids)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
            pred_classes = torch.argmax(outputs, dim=1)
            true_classes = torch.argmax(labels, dim=1)
            train_total += labels.size(0)
            train_correct += (pred_classes == true_classes).sum().item()
            
            train_progress.set_postfix({'loss': train_loss / (train_progress.n + 1), 'accuracy': 100 * train_correct / train_total})
        
        avg_train_loss = train_loss / len(train_loader)
        train_accuracy = 100 * train_correct / train_total
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_accuracy)
        
        # validate
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            val_progress = tqdm(val_loader, desc=f'Epoch {epoch+1}/{epochs} [Valid]')
            
            for batch in val_progress:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                token_type_ids = batch['token_type_ids'].to(device)
                labels = batch['label'].to(device)
                
                outputs = model(input_ids, attention_mask, token_type_ids)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                pred_classes = torch.argmax(outputs, dim=1)
                true_classes = torch.argmax(labels, dim=1)
                val_total += labels.size(0)
                val_correct += (pred_classes == true_classes).sum().item()
                
                val_progress.set_postfix({'loss': val_loss / (val_progress.n + 1), 'accuracy': 100 * val_correct / val_total})
        
        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * val_correct / val_total
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(val_accuracy)
        
        print(f'Epoch {epoch+1}/{epochs}:')
        print(f'  Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')
        print(f'  Valid Loss: {avg_val_loss:.4f}, Valid Accuracy: {val_accuracy:.2f}%')
        
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            torch.save(model.state_dict(), 'best_bert_caption_classifier.pt')
            print(f'  New best model saved with validation accuracy: {val_accuracy:.2f}%')

### Defining the evaluation function (testing)


#### ***evaluate_classifier (function)***
Evaluate the classification model.
- model (nn.Module): Trained model.
- dataloader (DataLoader): Test data loader.
- device (str): 'cuda' or 'cpu'.

##### Returns
dict: Precision, Recall and F1 scores for the test set.


In [8]:
def evaluate_classifier(model, dataloader, device):
    model.eval()
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc='Evaluating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(input_ids, attention_mask, token_type_ids)
            
            pred_classes = torch.argmax(outputs, dim=1)
            true_classes = torch.argmax(labels, dim=1)
            
            all_preds.extend(pred_classes.cpu().numpy())
            all_labels.extend(true_classes.cpu().numpy())
    
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
    
    results = {
        'precision': precision,
        'recall': recall,
        'f1': f1
    }
    
    print(classification_report(all_labels, all_preds, target_names=['Model A', 'Model B']))
    
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Model A', 'Model B'], 
                yticklabels=['Model A', 'Model B'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.savefig('confusion_matrix.png')
    plt.close()
    
    return results

def plot_training_history(history):
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['val_acc'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()

In [9]:
print("Loading dataset")
df = pd.read_csv(DATA_PATH)
print(f"Loaded dataset with {len(df)} examples")

print(f"Label distribution: {df['model_label'].value_counts().to_dict()}")
print(f"Perturbation percentage distribution: {df['perturbation_percentage'].value_counts().to_dict()}")

# print("Loading BERT tokenizer")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

dataset = CaptionDataset(df, tokenizer)

# Split dataset into train, validation, and test sets (70:10:20)
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.10 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size],generator=torch.Generator().manual_seed(42))

print(f"Dataset split: Train={train_size}, Validation={val_size}, Test={test_size}")

batch_size = 32  # this gave a peak GPU memory utilization of around 13.6 GB during training
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

model = CaptionClassifier().to(device)

optimizer = optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.BCELoss()  # using binary cross-entropy

print("Training model...")
num_epochs = 3
train_classifier(model, train_loader, val_loader, optimizer, criterion, device, epochs=num_epochs)

plot_training_history(history)

print("Loading best model for evaluation...")
model.load_state_dict(torch.load('best_bert_caption_classifier.pt'))

print("Evaluating model on test set...")
results = evaluate_classifier(model, test_loader, device)

print("\nTest Results:")
print(f"Precision: {results['precision']:.4f}")
print(f"Recall: {results['recall']:.4f}")
print(f"F1 Score: {results['f1']:.4f}")

torch.save({'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'tokenizer': tokenizer.__class__.__name__, 'results': results}, 'bert_caption_classifier_model.pt')

print("Model training and evaluation complete. Model saved to 'bert_caption_classifier_model.pt'")

Loading dataset
Loaded dataset with 5568 examples
Label distribution: {'B': 2784, 'A': 2784}
Perturbation percentage distribution: {50: 1856, 10: 1856, 80: 1856}
Dataset split: Train=3897, Validation=556, Test=1115
Training model...


Epoch 1/3 [Valid]: 100%|██████████| 18/18 [00:17<00:00,  1.04it/s, loss=0.0203, accuracy=99.8]


Epoch 1/3:
  Train Loss: 0.1885, Train Accuracy: 93.69%
  Valid Loss: 0.0203, Valid Accuracy: 99.82%
  New best model saved with validation accuracy: 99.82%


Epoch 2/3 [Valid]: 100%|██████████| 18/18 [00:17<00:00,  1.03it/s, loss=0.0134, accuracy=99.8]


Epoch 2/3:
  Train Loss: 0.0157, Train Accuracy: 99.77%
  Valid Loss: 0.0134, Valid Accuracy: 99.82%


...

Epoch 3/3:
  Train Loss: 0.0153, Train Accuracy: 99.69%
  Valid Loss: 0.0132, Valid Accuracy: 99.82%
Loading best model for evaluation...


/tmp/ipykernel_31/618418794.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_bert_caption_classifier.pt'))


Evaluating model on test set...


Evaluating: 100%|██████████| 35/35 [00:34<00:00,  1.00it/s]


              precision    recall  f1-score   support

     Model A       1.00      1.00      1.00       551
     Model B       1.00      1.00      1.00       564

    accuracy                           1.00      1115
   macro avg       1.00      1.00      1.00      1115
weighted avg       1.00      1.00      1.00      1115


Test Results:
Precision: 0.9973
Recall: 0.9973
F1 Score: 0.9973
Model training and evaluation complete. Model saved to 'bert_caption_classifier_model.pt'
